In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pytz
import os
import requests
from bs4 import BeautifulSoup
import calendar
from dateutil.relativedelta import relativedelta
import re
import time
from langchain_community.document_loaders import PyPDFLoader

# Data Import

In [2]:
# Load the data from XLSX
airline_financials = pd.read_excel("airline_financial_data.xlsx", sheet_name="airline_financials") # primary financial data and metrics

In [3]:
# Define the list of airlines
airlines = sorted(airline_financials["Airline"].unique())
airlines

['AAL', 'DAL', 'LUV', 'UAL']

In [4]:
# Define the list of years
years = sorted(airline_financials["Year"].unique())
years

[2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]

In [5]:
# Define the list of periods
periods = sorted(airline_financials["Quarter"].apply(lambda x: f"Q{x}" if x != "FY" else x).unique())
periods

['FY', 'Q1', 'Q2', 'Q3', 'Q4']

# Scraping SEC Filings

## Global Variables and Functions

Define source URLs and html elements for locating the SEC filings

In [6]:
sec_filings_url = {"AAL": "https://americanairlines.gcs-web.com/sec-filings", "DAL": "https://ir.delta.com/financials/default.aspx#sec", "UAL": "https://ir.united.com/financial-performance/sec-filings"}
html_doc_base_url = {"AAL": "https://americanairlines.gcs-web.com/node", "DAL": None, "UAL": "https://ir.united.com/node"}
pdf_doc_base_url = {"AAL": "https://americanairlines.gcs-web.com/static-files", "DAL": "https://d18rn0p25nwr6d.cloudfront.net/CIK-0000027904", "UAL": "https://ir.united.com/static-files"}
container = {"AAL": "table", "DAL": "div", "UAL": "table"}
container_class = {"AAL": "nirtable", "DAL": "module-container", "UAL": "nirtable"}
filing_group_class = {"AAL": "views-field views-field-field-nir-sec-form", "DAL": None, "UAL": "views-field views-field-field-nir-sec-form"}

Define start and end dates for the period in which to scrape filings

In [7]:
# Define function to create start and end dates
def define_period_dates(year, period):
    # Create date components based on selected year and period
    if period == "FY":
        start_month = 1
        end_month = 12
    else:
        end_month = int(period[-1]) * 3
        start_month = end_month - 2
    start_day = 1
    end_day = calendar.monthrange(year, end_month)[1]
    # Create start and end date variables to constrain document scraping
    start_date = datetime(year, start_month, start_day)
    if period=="FY":
        end_date = (datetime(year, end_month, end_day) + relativedelta(months=2)) # add two months to the end date to capture annual filings that are released up to two months after end of period
    else:
        end_date = (datetime(year, end_month, end_day) + relativedelta(months=1) + relativedelta(days=1)) # add one month to the end date to capture quarterly filings that are released up to a month after end of period
    
    #Print messages when testing function operation
    #print(f"Start Date: {start_date}\nEnd Date: {end_date}")
    
    return start_date, end_date

In [8]:
# If manually defining the entire period, uncomment the following lines:
#start_date = datetime(2014, 1, 1)
#end_date = datetime.now()
#print("Start date:", start_date)
#print("End date:", end_date)

Define function to extract document links from a page and filter by date

In [9]:
def extract_filing_links(url, doc_base_url, container, container_class, filing_group_class, start_date, end_date, reached_start_date):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
        
    # Find the table containing the document links and their filing dates
    table = soup.find(container, {"class": container_class})  # Identify table element on page where document entries and links are stored
    rows = table.find_all("tr") # Identify all rows, where each row is a document entry with a filing date and html link
    page_links = [] # Initiate the document container list
    skip_count = 0 # Initiate a counter for number of documents skipped during scraping
    retrieved_count = 0 # Initiate a counter for number of documents retrieved during scraping
    target_filings = {"8-K", "10-K", "10-Q"} # Limit documents to main reports to minimize the amount of text retrieved
    
    for row in rows:
        # Extract the date and link from each row
        time_element = row.find("time", class_="datetime")  # Find <time> element with the class "datetime"
        filing_group = row.find("td", class_ = filing_group_class) # Find the Filing Group column
        links = row.find_all("a", href=True) # Find the document link element
        
        if time_element and filing_group and links:
            # Extract the filing date from the "datetime" attribute of the document table
            filing_date_str = time_element["datetime"]  # Get the full date string (e.g., "2024-11-19T05:00:00Z")      
            try:
                # Parse the filing date (from datetime attribute)
                filing_date = datetime.strptime(filing_date_str, "%Y-%m-%dT%H:%M:%SZ")  # Parse full datetime
            except ValueError:
                continue  # Skip if the date format is invalid

            # Extract Filing Group text from the filing_group attribute of the document table
            filing_group_div = filing_group.find("a") 
            filing_group_text = filing_group_div.get_text(strip=True) if filing_group_div else ""

            # Find the correct link in the row for the html version of the document (matching "/node/#####/html")
            filing_link = None
            doc_term = doc_base_url.rsplit("/", 1)[-1]
            for link in links:
                if re.match(f"^/{doc_term}/", link["href"]):
                    filing_link = link["href"]
                    break  # Stop checking once we find the correct link         
            if filing_link:
                full_filing_link = f"{doc_base_url.rsplit("/", 1)[0]}{filing_link}"  # Convert to absolute URL

            # If the filing date is within the range, retrieve the document link. If document is before start date, end scraping. Otherwise, skip document link and continue scraping.
            if start_date <= filing_date <= end_date and filing_group_text in target_filings:
                reached_start_date=False
                retrieved_count += 1 # increment count when a document is retrieved
                page_links.append(full_filing_link) # add link to list
            elif filing_date < start_date:
                reached_start_date=True
                break # End loop once beyond oldest document
            else:
                reached_start_date=False
                skip_count +=1 # increment count when a document is skipped

            print(full_filing_link)

    return page_links, reached_start_date

Define function to extract and scrape pages with date filtering

In [10]:
def scrape_filing_pages(airline, year, period, sec_filings_url, doc_base_url, container, container_class, filing_group_class):
    if airline not in ["AAL", "UAL"]:
        print(f"Cannot scrape filings for {airline}.")
        return
    
    else:
        # Set up pass through variables
        current_url = sec_filings_url[airline]
        doc_base_url = doc_base_url[airline]
        container = container[airline]
        container_class = container_class[airline]
        filing_group_class = filing_group_class[airline]
        all_links = []
        
        # Define start and end dates
        start_date, end_date = define_period_dates(year, period)

        # Initialize reached_start_date
        reached_start_date=False
        
        while current_url:
            
            # Print messages when testing function operation
            print(f"Scraping page: {current_url}")
            
            # Extract links from the current page with date filtering
            page_links, reached_start_date = extract_filing_links(current_url, doc_base_url, container, container_class, filing_group_class, start_date, end_date, reached_start_date)
            all_links.extend(page_links)
            
            # Find the "Next" button to continue paging
            response = requests.get(current_url)
            soup = BeautifulSoup(response.text, "html.parser")
            next_button = soup.find("a", href=True, rel="next")
            if next_button:
                # Construct the next page URL
                next_page = next_button['href']
                current_url = f"{sec_filings_url[airline]}{next_page}"  # Complete the URL
            else:
                # No "Next" button, stop paging
                print("No next page found, ending scrape.")
                current_url = None
            
            # Break loop once start date is reached
            if reached_start_date==True:
                break        

        # Display links to filings retrieved
        print(f"\nRetrieved {len(all_links)} filing documents for the time period:")
        #for link in all_links:
        #    print(link)
        
        return all_links

Define function to extract documents and metadata

In [11]:
import json

def extract_filings(pdfs, airline, year, period):

    filings = [] # initialize the filing list
    metadatas = [] # initialize the metadata list

    filing_counter = 0 # initialize the filing counter (counts number of filings)
    doc_counter = 0 # initialize the doc counter (counts number of filing pages)

    #Load the PDF documents and extract metadata
    for pdf in pdfs:
        loader = PyPDFLoader(pdf)
        documents = loader.load()

        # Extract document text and metadata
        for doc in documents:
            print(f"Processing {doc.metadata.get('title', '')}-page-{doc.metadata.get('page_label', 'Unknown')} from filing {filing_counter+1} of {len(pdfs)}.")
            doc_id = f"{airline}{year}{period}-{doc_counter}-{doc.metadata.get('title', f'Filing {filing_counter+1} of {len(pdfs)}')}-Page {doc.metadata.get('page_label', 'Unknown')} of {doc.metadata.get('total_pages', 'Unknown')}"
            text = doc.page_content
            metadata = str(doc.metadata)

            # Store in the filings and metadata lists
            filings.append({
                "id": doc_id,
                "airline": airline,
                "title": doc.metadata.get('title', ''),
                "form": doc.metadata.get('subject', '').split(" ", 1)[0],  # Extract the form type from the subject
                "date_filed": doc.metadata.get('creationdate', ''),
                "page_label": doc.metadata.get('page_label', 'Unknown'),
                "total_pages": doc.metadata.get('total_pages', 'Unknown'),
                "source": doc.metadata.get('source', ''),
                "text": text,
            })
            #metadatas.append({
            #    "id": doc_id,
            #    "airline": airline,
            #    "metadata": metadata
            #})

            doc_counter += 1 # increment the doc counter

        filing_counter += 1 # increment the filing counter

    # Save filings to JSON file
    output_dir = "SEC_Filings"
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, f"{airline}{year}{period}_filings.json")
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(filings, f, ensure_ascii=False, indent=2)

    print(f"Extracted {len(filings)} pages from {len(pdfs)} filings for {airline} in {year}{period}.")

In [12]:
import ast

def inspect_metadata(metadata):
    no_title_count = 0
    for meta in metadata:
        metadata_dict = ast.literal_eval(meta["metadata"])
        if not metadata_dict.get("title"):
            no_title_count += 1

    percent_no_title = (no_title_count / len(metadata)) * 100
    print(f"Percent without title: {percent_no_title:.2f}%")

    no_subject_count = 0
    for meta in metadata:
        metadata_dict = ast.literal_eval(meta["metadata"])
        if not metadata_dict.get("subject"):
            no_subject_count += 1

    percent_no_subject = (no_subject_count / len(metadata)) * 100
    print(f"Percent without subject: {percent_no_subject:.2f}%")

    no_creationdate_count = 0
    for meta in metadata:
        metadata_dict = ast.literal_eval(meta["metadata"])
        if not metadata_dict.get("creationdate"):
            no_creationdate_count += 1

    percent_no_creationdate = (no_creationdate_count / len(metadata)) * 100
    print(f"Percent without creationdate: {percent_no_creationdate:.2f}%")

Define a combined function to complete the scraping of SEC filings for any number of airlines, years, and periods

In [13]:
def filing_scraping_pipeline(airlines, years, periods, sec_filings_url, pdf_doc_base_url, container, container_class, filing_group_class):
    for airline in airlines:
        for year in years:
            for period in periods:
                print(f"Scraping filings for {airline} in {year}{period}...")
                # Step 1: Scrape the investor relations page for the airline to get the filing PDF links
                links = scrape_filing_pages(airline, year, period, sec_filings_url, pdf_doc_base_url, container, container_class, filing_group_class)
                # Step 2: Extract the filings from the PDF document links
                extract_filings(links, airline, year, period)


## Scrape Filings

Define the variables to parse.

In [14]:
scraping_airlines = ["AAL"]  # This will loop through the selected airlines. Can be changed to any combination of airline(s) in the dataset ("AAL", "DAL", "LUV", or "UAL").
scraping_years = [2025] # This will loop through the selected years. Can be changed to any combination of year(s) in the dataset (2014 to present).
scraping_periods = ["Q3"]  # This will loop through the selected periods. Can be changed to any combination of periods in the dataset ("FY", "Q1", "Q2", "Q3", "Q4").

**RUN THE FILING SCRAPING PIPELINE**

In [15]:
%%time
filing_scraping_pipeline(scraping_airlines, scraping_years, scraping_periods, sec_filings_url, pdf_doc_base_url, container, container_class, filing_group_class)

Scraping filings for AAL in 2025Q3...
Scraping page: https://americanairlines.gcs-web.com/sec-filings
https://americanairlines.gcs-web.com/static-files/6bf50f54-07df-4171-92bb-a78f9acea628
https://americanairlines.gcs-web.com/static-files/933aba3f-93fd-46a6-9181-89e4840e57c8
https://americanairlines.gcs-web.com/static-files/c2a093f2-7ccb-4782-8c00-3223a1966919
https://americanairlines.gcs-web.com/static-files/753314c0-549d-483b-b519-d65c0954e89a
https://americanairlines.gcs-web.com/static-files/12b2604e-ce9d-417c-ab23-1f6b39df4efe
https://americanairlines.gcs-web.com/static-files/a7f3103d-0b3a-4246-b0f8-295878665327
https://americanairlines.gcs-web.com/static-files/743e8a51-c11b-4d71-a577-34a2f6ee9b2c
https://americanairlines.gcs-web.com/static-files/7fc2e962-4109-4cb0-9a13-1cab48515b43
https://americanairlines.gcs-web.com/static-files/77fc1bc9-631c-4fb9-921e-ec8d94264c7d
https://americanairlines.gcs-web.com/static-files/97922c1e-d84a-485d-871b-6985f583fef4
Scraping page: https://ameri